In [18]:
! git clone https://github.com/ArcInstitute/state.git
%cd state

# Colab-specific config for pytorch lightning
import os
os.environ['MPLBACKEND'] = 'Agg'

Cloning into 'state'...
remote: Enumerating objects: 8428, done.
remote: Counting objects: 100% (1833/1833), done.
remote: Compressing objects: 100% (380/380), done.
remote: Total 8428 (delta 1638), reused 1453 (delta 1453), pack-reused 6595 (from 2)
Receiving objects: 100% (8428/8428), 126.02 MiB | 23.41 MiB/s, done.
Resolving deltas: 100% (5274/5274), done.
/content/VCC-state/state/state


In [22]:
!ls

assets	  MODEL_ACCEPTABLE_USE_POLICY.md  README.md  singularity.def
examples  MODEL_LICENSE.md		  ruff.toml  src
LICENSE   pyproject.toml		  scripts    tests


In [2]:
from google.colab import drive
drive.mount('/content/drive')

%pip install tqdm

from tqdm.auto import tqdm  # picks the best bar for the environment

output_path = "/content/drive/MyDrive/VCC/datasets/competition_support_set.zip"

from zipfile import ZipFile
from tqdm.auto import tqdm

out_dir  = "competition_support_set"

os.makedirs(out_dir, exist_ok=True)

with ZipFile(output_path, 'r') as z:
    for member in tqdm(z.infolist(), desc="Unzipping", unit="file"):
        z.extract(member, out_dir)

!ls competition_support_set

Mounted at /content/drive


Unzipping:   0%|          | 0/10 [00:00<?, ?file/s]

In [11]:
!ls competition_support_set

competition_train.h5	       gene_names.csv  k562_gwps.h5  starter.toml
competition_val_template.h5ad  hepg2.h5        k562.h5
ESM2_pert_features.pt	       jurkat.h5       rpe1.h5


In [12]:
entity = "arcinstitute" # @param {"type":"string","placeholder":"arcinstitute"}
! sed -i 's|entity: your_entity_name|entity: {entity}|g' VCC-state/state/src/state/configs/wandb/default.yaml

sed: can't read VCC-state/state/src/state/configs/wandb/default.yaml: No such file or directory


In [4]:
! uv run state --help

Using CPython 3.11.14
Creating virtual environment at: .venv
Installed 152 packages in 1.91s
usage: state [-h] {emb,tx} ...

positional arguments:
  {emb,tx}

options:
  -h, --help  show this help message and exit


In [7]:
! uv run state tx train \
  data.kwargs.toml_config_path="competition_support_set/starter.toml" \
  data.kwargs.num_workers=8 \
  data.kwargs.batch_col="batch_var" \
  data.kwargs.pert_col="target_gene" \
  data.kwargs.cell_type_key="cell_type" \
  data.kwargs.control_pert="non-targeting" \
  data.kwargs.perturbation_features_file="competition_support_set/ESM2_pert_features.pt" \
  training.max_steps=20 \
  training.ckpt_every_n_steps=10 \
  model=state_sm \
  wandb.tags="[small_run]" \
  wandb.project=vcc \
  wandb.entity=arcinstitute \
  output_dir="competition" \
  name="small_run"

Traceback (most recent call last):
  File "/content/VCC-state/state/.venv/bin/state", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/content/VCC-state/state/src/state/__main__.py", line 120, in main
    cfg = load_hydra_config("tx", args.hydra_overrides)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/VCC-state/state/src/state/__main__.py", line 45, in load_hydra_config
    cfg = compose(config_name="config", overrides=overrides)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/VCC-state/state/.venv/lib/python3.11/site-packages/hydra/compose.py", line 38, in compose
    cfg = gh.hydra.compose_config(
          ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/VCC-state/state/.venv/lib/python3.11/site-packages/hydra/_internal/hydra.py", line 594, in compose_config
    cfg = self.config_loader.load_configuration(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/VCC-state/state/.venv/lib/python3.11/site

In [17]:
!ls src/state/configs/wandb

config.yaml  data  __init__.py	model  state-defaults.yaml  training
